In [1]:
import pyspark.sql.functions as F
from data_location import DELTA_LOCATION

from spark_bi.spark import FutPathlingContext

pc = FutPathlingContext.create(app_name="example-spark-app")
delta_lake = pc.read.delta(DELTA_LOCATION)

:: loading settings :: url = jar:file:/Users/mabe/Git/spark-bi/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/mabe/.ivy2.5.2/cache
The jars for the packages stored in: /Users/mabe/.ivy2.5.2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-34f8d52e-58de-441c-a0ed-bed9b3cad9a1;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;9.1.0 in local-m2-cache
	found io.delta#delta-spark_2.13;4.0.0 in local-m2-cache
	found io.delta#delta-storage;4.0.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.13.1 in local-m2-cache
:: resolution report :: resolve 88ms :: artifacts dl 4ms
	:: modules in use:
	au.csiro.pathling#library-runtime;9.1.0 from local-m2-cache in [default]
	io.delta#delta-spark_2.13;4.0.0 from local-m2-cache in [default]
	io.delta#delta-storage;4.0.0 fr

In [2]:
careteams = delta_lake.view(
    resource="CareTeam", select=[{"column": [{"name": "careteam_id", "path": "getResourceKey()"}]}]
)
careteams.head(5)

25/12/12 11:25:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(careteam_id='CareTeam/3000000001'),
 Row(careteam_id='CareTeam/3000000002'),
 Row(careteam_id='CareTeam/3000000005'),
 Row(careteam_id='CareTeam/3000000006'),
 Row(careteam_id='CareTeam/3000000007')]

In [3]:
episodes_of_care = delta_lake.view(
    resource="EpisodeOfCare",
    select=[
        {
            "column": [
                {"name": "episodeofcare_id", "path": "getResourceKey()"},
                {"name": "patient_id", "path": "patient.getReferenceKey()"},
                {
                    "name": "organization_id",
                    "path": "managingOrganization.first().getReferenceKey()",
                },
            ]
        },
        {"forEach": "team", "column": [{"name": "team_id", "path": "getReferenceKey()"}]},
    ],
)
episodes_of_care.head(5)

[Row(episodeofcare_id='EpisodeOfCare/2000000029', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752'),
 Row(episodeofcare_id='EpisodeOfCare/2000000035', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752'),
 Row(episodeofcare_id='EpisodeOfCare/2000000042', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752'),
 Row(episodeofcare_id='EpisodeOfCare/2000000049', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752'),
 Row(episodeofcare_id='EpisodeOfCare/2000000068', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752')]

In [4]:
from shlex import join


joined = (
    careteams.join(episodes_of_care, careteams.careteam_id == episodes_of_care.team_id, how="left")
    .select("team_id", "patient_id")
    .distinct()
    .groupBy("team_id")
    .agg(F.count("*").alias("n_patients"))
    .sort("n_patients", ascending=False)
)
joined.head(5)

[Row(team_id='CareTeam/3000143203', n_patients=3),
 Row(team_id='CareTeam/3000000002', n_patients=2),
 Row(team_id='CareTeam/3000178177', n_patients=1),
 Row(team_id=None, n_patients=1)]